In [2]:
# import the wrapper
from Leaf import *

In [3]:
# The wrapper can be either initialized by a parameter file (legacy) 
# or given the parameters directly as dict (recommended).
# Parameters not given are automatically set to default

# An initialization of the wrapper may look like this (remove anything not needed)
# The given values repres

# set a path where the data will be saved
data_path = "./data/" # <- don't forget the "/" at the end
# set a prefix for the saved files
data_prefix = "run_" # results in files run_{run_id}.h5
# set a path for the cache, if None, write no cache (recommended for sampling)
cache_path = None

# see what is required for the Leaf class
Leaf?

Init signature:
Leaf(
    data_path: str = './data/',
    data_prefix: str = 'simrun_',
    parameter_file: str = None,
    cache_path: str = None,
    debug: bool = False,
    redshift: float = None,
    astro_params: dict = {},
    cosmo_params: dict = {},
    user_params: dict = {},
    flag_options={},
    global_params: dict = {},
) -> None
Docstring:      <no docstring>
Init docstring:
Initializes the Leaf class with specified parameters.

Args:
    data_path (str): The path where output will be saved. Default is "./data/"

    data_prefix (str): The prefix for the output files saved in the specified data_path. Default is "simrun_".
    
    parameter_file (str): The file from which to load parameters for all runs. This allows for more control over specific parameters.
                          If None, standard parameters are used. Default is None.
    
    cache_path (str): Path for the 21cmFAST cache. If None, don't write cache. This can be faster if fast IO is available, but 

In [ ]:
# Initialize the simulator (Leaf class) either by passing a path as the argument parameter_path
# or initialize it via dicts as shown below

# define hyperparameter of the simulation
user_parameter = {
    "HII_DIM": 140,
    "BOX_LEN": 200,
    "N_THREADS": 1,
    "USE_INTERPOLATION_TABLES": True,
    "PERTURB_ON_HIGH_RES": True
}
flag_options = {
    "INHOMO_RECO": True,
    "USE_TS_FLUCT": True
}

astro_params = {
    "INHOMO_RECO": True
}

redshift = 5.5

# define saving behavoir 
data_path = "./data/"
data_prefix = "run_" # results in files like run_0, run_1, ...

# initialize the simulation class
sim = Leaf(user_params=user_parameter, flag_options=flag_options, astro_params=astro_params, debug=True, redshift=redshift
           data_path=data_path=, data_prefix=data_prefix)

In [3]:
# To just run a single box using the parameters above one can just 
# the bool argument save determines if the result will be returned or saved on disk
box = sim.run_box(save=False)

# one could also pass different arguments for the specific run
# e.g. one is interested in how the box may look like if
astro_params = {
    "L_X": 41
}

box_different_L_X = sim.run_box(save=False, astro_params=astro_params)

In [8]:
# To simulate a lightcone, one can do
lc = sim.run_lightcone(save=False)
# The parametes for the run_lightcone method are the same as for the run_box
# except:
# There is also a sanity check (5 sigma from the Planack observation) activatable by filter_peculiar
# And a sanity check to get rid of NaNs (which may occur) via NN interpolation by activating sanity_check

In [ ]:
# Again, parameters can be changed on-the-fly
lc_different_L_X = sim.run_lightcone(save=False, astro_params=astro_params)

In [ ]:
# One can also run the simulator on a prior distributior
# This is useful to create a database
# For cluster deployment, change the threads argument to the number of available cpu cores

# we first need to define parameter ranges for the parameters of interest
# e.g. we are interested in the HII_EFF_FACTOR and OMm

astro_params_range = {
    "HII_EFF_FACTOR": [20,40]
}

cosmo_params_range = {
    "OMm": [0.1,0.3]
}

# we also need to define a samplefunction. The arrays of the ranges are the parameters passed to this samplefunction
# Currently there are 3 functions implemented: uniform, gauss, gumble
# We can define the samplefunction to be uniform by e.g. (or define any custom function)
samplef = Leaf.uniform

# don't forget to add when running with more than 1 thread (multiprocessing may cause problems in jupyter notebooks)
#if __name__ == '__main__':
sim.run_lcsampling(save=False, threads=1, quantity=4, astro_params_range=astro_params_range,
                   cosmo_params_range=cosmo_params_range)

In [ ]:
# A full pipeline may look like this

from Leaf import *
cosmo_params = {
"OMm": 0.2
}
astro_params = {
"L_X": 40,
"NU_X_THRESH": 100,
"ION_Tvir_MIN": 4.5,
"HII_EFF_FACTOR": 29,
"INHOMO_RECO": True
}
global_params = {
    "M_WDM": 4
}
user_params = {
"HII_DIM": 40,
"BOX_LEN": 160,
"N_THREADS": 2,
"USE_INTERPOLATION_TABLES": True,
"PERTURB_ON_HIGH_RES": True
}

flag_options = {
"INHOMO_RECO": True,
"USE_TS_FLUCT": True
}

sim = Leaf(debug=True)



testsim = sim.run_lightcone(save=False, redshift=5., filter_peculiar=False,
                            cosmo_params=cosmo_params, astro_params=astro_params,
                            global_params=global_params, user_params=user_params,
                            flag_options=flag_options)


if __name__ == '__main__':
    sim.run_lcsampling(samplef=sim.uniform, redshift=5.5, save=True, threads=2, quantity=4,
                    astro_params_range=astro_params_range)

Don't forget to check the docs or type e.g. sim.run_lcsampling? if you feel lost.